# The Natário Warp Drive

This notebook explores the Natário expansion-free warp drive.

**Reference**: Natário, J. (2002). Class. Quantum Grav. 19, 1157.

## Key Innovation

Natário constructed a warp drive with **zero expansion/contraction** by requiring the shift vector to be divergence-free:

$$\nabla \cdot \vec{\beta} = \partial_i \beta^i = 0$$

This eliminates the characteristic expansion (behind) and contraction (in front) regions of the Alcubierre metric.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from warpbubblesim.metrics import AlcubierreMetric, NatarioMetric
from warpbubblesim.gr import compute_energy_density, check_energy_conditions
from warpbubblesim.gr.adm import compute_expansion_scalar
from warpbubblesim.gr.geodesics import integrate_geodesic
from warpbubblesim.viz.fields2d import plot_energy_density, plot_expansion_scalar, plot_shift_vector_field

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

## 1. The Natário Metric

The metric still has ADM form with $\alpha = 1$ and flat spatial metric, but the shift vector is constructed differently to satisfy the divergence-free condition.

In [ ]:
# Create Natário metric
natario = NatarioMetric(v0=1.0, R=1.0, sigma=8.0)

# Display info
info = natario.info()
print(f"Metric: {info['name']}")
print(f"Citation: {info['citation']}")
print(f"Parameters: {info['parameters']}")

## 2. Verify Divergence-Free Condition

The key property of the Natário shift vector:

In [ ]:
# Check divergence at various points
test_points = [
    (0, 0, 0, 0),
    (0, 1.0, 0.5, 0),
    (0, 0.5, 0.3, 0.2),
    (0, 2.0, 1.0, 0.5),
]

print("Divergence of shift vector (should be near zero):")
for t, x, y, z in test_points:
    div = natario.verify_divergence_free(t, x, y, z)
    print(f"  ({x}, {y}, {z}): div(beta) = {div:.2e}")

## 3. Compare with Alcubierre

### Shift Vector Field

In [ ]:
alcubierre = AlcubierreMetric(v0=1.0)
natario = NatarioMetric(v0=1.0)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, metric, title in [(axes[0], alcubierre, 'Alcubierre'),
                           (axes[1], natario, 'Natário')]:
    plot_shift_vector_field(metric, x_range=(-3, 3), y_range=(-3, 3),
                            nx=15, ny=15, ax=ax)
    ax.set_title(f'{title} Shift Vector')

plt.tight_layout()
plt.show()

### Expansion Scalar

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, metric, title in [(axes[0], alcubierre, 'Alcubierre'),
                           (axes[1], natario, 'Natário')]:
    plot_expansion_scalar(metric, x_range=(-3, 3), y_range=(-3, 3),
                          nx=80, ny=80, ax=ax)
    ax.set_title(f'{title} Expansion Scalar')

plt.tight_layout()
plt.show()

print("\nNote: Natário metric has zero expansion everywhere (uniform color)")

### Energy Density

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, metric, title in [(axes[0], alcubierre, 'Alcubierre'),
                           (axes[1], natario, 'Natário')]:
    plot_energy_density(metric, x_range=(-3, 3), y_range=(-3, 3),
                        nx=80, ny=80, ax=ax)
    ax.set_title(f'{title} Energy Density')

plt.tight_layout()
plt.show()

## 4. Energy Conditions

Despite the different construction, Natário still requires negative energy:

In [ ]:
natario = NatarioMetric(v0=1.0)
metric_func = natario.get_metric_func()

# Check conditions in the wall
coords = np.array([0, 1.0, 0.3, 0])
conditions = check_energy_conditions(metric_func, coords)

print("Energy conditions for Natário (wall region):")
for name, (satisfied, value) in conditions.items():
    status = "SATISFIED" if satisfied else "VIOLATED"
    print(f"  {name}: {status} (value: {value:.2e})")

## 5. Geodesics Comparison

In [ ]:
def plot_geodesics_comparison(metric1, metric2, name1, name2):
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    for ax, metric, name in [(axes[0], metric1, name1), (axes[1], metric2, name2)]:
        metric_func = metric.get_metric_func()
        
        for x0 in np.linspace(-3, 3, 7):
            initial_coords = np.array([0.0, x0, 0.0, 0.0])
            initial_velocity = np.array([1.0, 0.0, 0.0, 0.0])
            
            result = integrate_geodesic(
                metric_func, initial_coords, initial_velocity,
                lambda_span=(0, 10), max_step=0.1
            )
            coords = result['coords']
            ax.plot(coords[:, 1], coords[:, 0], alpha=0.7)
        
        # Bubble trajectory
        t_range = np.linspace(0, 10, 100)
        x_bubble = [metric.bubble_center(t) for t in t_range]
        ax.plot(x_bubble, t_range, 'k--', linewidth=2, label='Bubble')
        
        ax.set_xlabel('x')
        ax.set_ylabel('t')
        ax.set_title(f'{name} Geodesics')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

alcubierre = AlcubierreMetric(v0=1.0)
natario = NatarioMetric(v0=1.0)

fig = plot_geodesics_comparison(alcubierre, natario, 'Alcubierre', 'Natário')
plt.show()

## 6. Mathematical Construction

### Divergence-Free Shift

To construct a divergence-free shift, Natário uses a vector potential $\vec{A}$:

$$\vec{\beta} = \nabla \times \vec{A}$$

The curl of any vector field is automatically divergence-free:

$$\nabla \cdot (\nabla \times \vec{A}) = 0$$

### Resulting Shift Components

The specific construction yields shift components that differ from Alcubierre, particularly having non-zero $\beta^y$ and $\beta^z$ components.

In [ ]:
natario = NatarioMetric(v0=1.0)

# Compare shift components at a point
t, x, y, z = 0, 1.0, 0.5, 0.2

beta = natario.shift(t, x, y, z)
print(f"Natário shift at ({x}, {y}, {z}):")
print(f"  beta^x = {beta[0]:.4f}")
print(f"  beta^y = {beta[1]:.4f}")
print(f"  beta^z = {beta[2]:.4f}")

alcubierre = AlcubierreMetric(v0=1.0)
beta_alc = alcubierre.shift(t, x, y, z)
print(f"\nAlcubierre shift at ({x}, {y}, {z}):")
print(f"  beta^x = {beta_alc[0]:.4f}")
print(f"  beta^y = {beta_alc[1]:.4f}")
print(f"  beta^z = {beta_alc[2]:.4f}")

## 7. Parameter Study

In [ ]:
# Different velocities
velocities = [0.5, 1.0, 1.5, 2.0]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for ax, v in zip(axes, velocities):
    metric = NatarioMetric(v0=v)
    plot_energy_density(metric, x_range=(-3, 3), y_range=(-3, 3), 
                        nx=80, ny=80, ax=ax)
    ax.set_title(f'$v_s = {v}c$')

plt.suptitle('Natário Energy Density vs Velocity', fontsize=14)
plt.tight_layout()
plt.show()

## Summary

The Natário warp drive:

1. **Expansion-free**: No expansion/contraction regions ($\theta = 0$ everywhere)
2. **Still requires exotic matter**: Negative energy density persists
3. **Different energy distribution**: Shape differs from Alcubierre
4. **Mathematical elegance**: Divergence-free shift from curl construction
5. **Same physical limitations**: Energy conditions still violated

The lack of expansion regions might have implications for horizon formation and Hawking radiation, but the fundamental exotic matter requirement remains.